In [53]:
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import requests
import datetime
import time
import os
import json
import glob
import random

#declaring base open and colsed docket link
base_link = "https://www.gao.gov/search?f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01"

#declaring chrome header
hdr1 = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
	   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
	   'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
	   'Accept-Encoding': 'none',
	   'Accept-Language': 'en-US,en;q=0.8',
	   'Connection': 'keep-alive'}

log_file_name_with_path = "resume_helper_logs.txt"


def html_data_grabber(given_page_link,header = hdr1) : 
	""" 
		This function returns dict of information after browsing 
		any given link having soup content of page, status code,
		was it able to grab page data or not with "grabbed data" 
		key and additional helpful message of how url opening went. 
  
		Parameters: 
			given_page_link    (str)    :  url to browse
										   ex input : https://www.gao.gov/products/b-419833.2
			header             (dict)   :  dict of browser header information default is set to chrome
		Returns: 
			dict: dict of information after browsing any given link having soup content of page,
			status code,was it able to grab page data or not with "grabbed data" key and
			additional helpful message of how url opening went.
		"""
	request_made = requests.get(given_page_link,headers = header)
	print(given_page_link)

	#time.sleep(random.randint(3, 8)) #random sleep as per MS guidelines although recommended Crawl-delay on https://www.gao.gov/robots.txt is 10s

	status_code = int(request_made.status_code)
	data = ""
	grabbed_data = False
	if status_code == 200 :
		grabbed_data = True
		data = request_made.text
		message = "Success"
	elif status_code >= 403 :
		message = "banned"
	elif status_code >= 500 :
		message = "Government Server is down"
	else :
		message = "your internet connection is down"
		
	if grabbed_data :
		data = soup(data,"html.parser")
		
	return {"content":data, "status_code":status_code, "grabbed_data":grabbed_data, "message": message}
		
def available_search_result_page_links(base_link) :
	""" 
		This function returns list of all search result 
		pages link that can be browsed from base page
  
		Parameters: 
			base_link    (str)    :  base search page link 
									 ex input : https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1#s-skipLinkTargetForMainSearchResults
		Returns: 
			list: A list of all seach pages results links that can be browsed from base search page
				  Also will raise an error if it couldn't browse or open search page.
		"""
	base_link_content = html_data_grabber(base_link)
	if base_link_content["grabbed_data"] :
		last_page_link = "https://www.gao.gov/search?" + base_link_content["content"].findAll("a",{"title" : "Go to last page"})[0]["href"]
	else :
		print(base_link,base_link_content["message"],base_link_content["status_code"])
		raise

	last_page_no = int(last_page_link.split("&")[-1].replace("page=",""))
	all_links = [last_page_link.split("page=")[0] + "page=" + str(i) for i in range(0,last_page_no + 1)]
	return all_links

def docket_page_links_from_search_page(page_link) :
	""" 
		This function returns list of all docket page links 
		present on a open/closed docket search filter result.
  
		Parameters: 
			page_link    (str)    :  single docket search page link 
									 ex input : https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=82
		Returns: 
			list: A list of all docket page links present of a docket search page
				  Also will raise an error if it couldn't browse or open search page.
		"""
	browse_link = html_data_grabber(page_link)
	if browse_link["grabbed_data"] :
		docket_content = browse_link["content"].findAll("article",{"class" : "node node--type-product node--view-mode-teaser-search"})
	else :
		print(page_link,browse_link["message"],browse_link["status_code"])
		raise
	all_links = []
	base_link = "https://www.gao.gov"
	all_links = [base_link + i["about"] for i in docket_content]
	if len(all_links) == 0 :
		raise
	return all_links


def extract_data_from_docket_page(docket_page_link) :
	""" 
		This function returns dict of all docket information present
		on docket page link .
  
		Parameters: 
			docket_page_link    (str)    :  single docket page link 
											ex input : https://www.gao.gov/docket/b-418742.3
		Returns: 
			dict: A dictionary of all docket page information with key value pair
				  Also will raise an error if it couldn't browse or open docket link.
		"""
	browse_link = html_data_grabber(docket_page_link)
	if browse_link["grabbed_data"] :
		data = browse_link["content"].findAll("div",{"class" : "node__content"})[0]
	else :
		print(docket_page_link, browse_link["message"],browse_link["status_code"])
		raise
	all_data = {"docket_page_link" : docket_page_link}

	all_data["highlights_tags"] = str(data.findAll("div",{"class" : re.compile(r'^clearfix text-formatted field field--name-product-highlights')})[0])
	try :
		all_data["decision_tags"] = str(data.findAll("div",{"class" : re.compile(r'^js-endpoint-view-decision field field--name-field-h')})[0])
	except :
		all_data["decision_tags"] = ""
	all_data["page_title_or_company"] = browse_link["content"].findAll("section",{"id" : 'block-gao-page-title'})[0].text.strip()
	all_data["file_no_and_date"] = browse_link["content"].findAll("section",{"id" : 'block--post-title-info'})[0].text.strip()
	all_data["crawl_time"] =  str(datetime.datetime.now()).split(".")[0]
    

	if len(all_data) < 3 : # just checking if crawl went successsful and have valueselse will raise error
		raise
	return all_data
 
		
def all_docket_data_from_search_page_link(search_page_link,log_file_name_with_path = False) :
	""" 
		This function returns list of dict docket data for open/closed 
		docket search filter result with provided parameters. Also writes a log in
		form of links that it has visited if "log_file_name_with_path" variable 
		provided with log file path or file name
  
		Parameters: 
			search_page_link         (str)    : single docket search page link 
												ex input : https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1&page=82
			log_file_name_with_path  (str)    : initialize with Flase, takes input file path in form of string
												helpful while resuming when crawl interrupts midway
												ex input "resume_helper_logs.txt"(not required input though) 
		Returns: 
			list: A list of all docket data in dictionaries
		"""
	all_docket_data = []
	all_docket_page_links = docket_page_links_from_search_page(search_page_link)
	for single_docket in all_docket_page_links :
		if log_file_name_with_path :
			if not log_file_presence_checker(single_docket,log_file_name_with_path) :
				all_docket_data.append(extract_data_from_docket_page(single_docket))
			else :
				pass
		else :
			all_docket_data = [extract_data_from_docket_page(i) for i in all_docket_page_links]
	if log_file_name_with_path :
		write_log = log_file_writer(all_docket_page_links,log_file_name_with_path)
	return all_docket_data

#functions below to be used only in production

def log_file_presence_checker(link_to_check,log_file_name_with_path) :
	""" 
		This function checks if a links is present
		in log file,it's a helper for implenting crawl resume
  
		Parameters: 
			link_to_check               (str)  :  single link
			log_file_name_with_path     (str)  :  log filename to check 
		Returns: 
			Boolean : Returns either True or False based on if a given 
					  link is present in log file or not i.e indicating
					  if it is already browsed by crawler or not
		"""
	if not os.path.isfile(log_file_name_with_path) :
		return False
	with open(log_file_name_with_path,"r")as f :
		content = f.read()
	if link_to_check in content :
		return True
	else :
		return False
	
def log_file_writer(link_to_write,log_file_name_with_path) :
	""" 
		This function writes links as a log to given file.
		helps in resuming crawl
  
		Parameters: 
			link_to_write               (str/list)  :  single link or list of links
			log_file_name_with_path     (str)       :  log filename to write 
		Returns: 
			null
		"""
	with open(log_file_name_with_path,"a") as f :
		if isinstance(link_to_write, list) :
			for single_link in link_to_write :
				f.write(str(single_link) + "\n")
		else :
			f.write(str(link_to_write) + "\n")
		
def temp_file_writer(all_data,temp_file_with_path) :
	""" 
		This function writes current crawl data which are list of dict docket data
		provided by "temp_open_close_all_docket_data_downloader" function.
		This is a helper function and benefits in resuming crawl during interruptions.
  
		Parameters: 
			all_data                (list) :  list of dicts docket data
			temp_file_with_path     (str)  :  temp json store filename with path 
		Returns: 
			null
		"""
	with open(temp_file_with_path,"a") as f :
		for single_data in all_data :
			f.write(json.dumps(single_data) + "\n") 

def temp_open_close_all_docket_data_downloader(docket_type, base_open_close_link) :
	""" 
		This function first checks if a given docket type(open/close) crawl csv file has
		been created or not on todays date if so then it simply returns false (thus limiting only
		one complete crawl on a given day); if not then it takes base open/close search page
		result link and crawl each page result and every docket link present on them, grabs their
		information and write them to docket_type + "_temp_data.json" named json file. 
		But before it crawl any page it checks if that docket or search link is present
		in "log_file_name_with_path" or not. If it's present there then it wont crawl it,
		if not then proceed with the crawl and then write those links to log file after the
		successful crawl. This helps in resuming the crawl from where it left off if the
		crawling gets interrupted midway due to any issue for a given docket type.
		Log file and json file are created in current working directory
  
		Parameters: 
			docket_type           (str)    : takes docket type name either open or close
												ex input : "open" or "close"
			base_open_close_link  (str)    : base search page link for either open or close
											 docket search result
											 ex input : https://www.gao.gov/legal/bid-protests/search?processed=1&closed=1#s-skipLinkTargetForMainSearchResults 
		Returns: 
			str: Succesfully crawled temp json file name or False
				 if a final csv for the day is already generated.
		"""
	today = datetime.datetime.now().date()
	already_created_files = glob.glob(str(docket_type)+"*.csv")
	for single_file in already_created_files :
		date = re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2})", single_file)[0]
		date = datetime.datetime.strptime(date, '%Y-%m-%d')
		if date.date() == today :
				return False
			
	temp_file_name = str(docket_type) + "_temp_data.json"

	file_count = 0
	all_search_pages_results = available_search_result_page_links(base_open_close_link)
	for single_link in all_search_pages_results :
		print(single_link)
		if not log_file_presence_checker(single_link,log_file_name_with_path) :
			data = all_docket_data_from_search_page_link(single_link,log_file_name_with_path)
			temp_file_writer(data,temp_file_name)
			log_file_writer(str(single_link),log_file_name_with_path)

		file_count = file_count + 1 
		percent_calci = (float(file_count)/float(len(all_search_pages_results)))*100.0
		print ("working on link no " + str(file_count) + " of total links " + str(len(all_search_pages_results)) + " " + str(percent_calci) + " % completed")
	return temp_file_name
   
def final_csv_generator(docket_type,temp_file_name_with_path = False,writepath = False) :
	""" 
		This function reads temp json file provided by function
		"temp_open_close_all_docket_data_downloader";Should be used 
		only after above functions successful execution.This convert 
		the given json file "temp_file_name_with_path" to a pandas 
		dataframe and writes it to csv with filename having docket 
		type and system current time. Then deletes temporary json file 
		and current docket log file.
  
		Parameters: 
			docket_type               (str)          : takes docket type name either open or close
													   ex input : "open" or "close"
			temp_file_name_with_path  (str)          : temp json store filename with path return by func
													   "temp_open_close_all_docket_data_downloader
			writepath                 (str)          : currently not implemented 
		Returns: 
			pandas dataframe : Succesfully execution will return a pandas dataframe else 
							   will return null if final crawl csv file is present in root
							   directory.
		"""
	if not temp_file_name_with_path :
		return
	with open(temp_file_name_with_path,"r") as f :
		all_data_to_be_written = [json.loads(i) for i in f.readlines() if json.loads(i) != False]
	df_docket = pd.DataFrame(all_data_to_be_written)
	if "Decision Highlight" in df_docket : #just putting "Decision Highlight" column to last
		df1 = df_docket.pop('Decision Highlight')
		df_docket['Decision Highlight'] = df1
	#df_docket = df_docket.drop_duplicates()  
	current_date = str(datetime.datetime.now()).split(" ")[0]
	if writepath :
		df_docket.to_csv(writepath + docket_type +"_" + current_date + ".csv", index = False)
	else :
		df_docket.to_csv(docket_type + "_" + current_date + ".csv", index = False)
		
	if os.path.isfile(log_file_name_with_path) :
		os.remove(log_file_name_with_path)
	if os.path.isfile(temp_file_name_with_path) :
		os.remove(temp_file_name_with_path)
	return df_docket

# if __name__ == "__main__":

# 	#grabbing data of all open docket
# 	start_time = datetime.datetime.now() 
# 	print("grabbing data for open docket")
# 	temp_file_name_with_path = temp_open_close_all_docket_data_downloader("open_docket", open_docket_base_link)
# 	df = final_csv_generator("open_docket",temp_file_name_with_path)
# 	print("completed")
# 	if (datetime.datetime.now() - start_time).seconds < 5 :
# 		print("Already crawled todays file, only one time crawl is allowed in a day")
# 	print("Execution time = %s" % (datetime.datetime.now() - start_time)) 



# 	#grabbing data of all close docket
# 	start_time = datetime.datetime.now() 
# 	print("grabbing data for close docket")
# 	temp_file_name_with_path = temp_open_close_all_docket_data_downloader("close_docket", closed_docket_base_link)
# 	df2 = final_csv_generator("close_docket",temp_file_name_with_path)
# 	print("completed")
# 	if (datetime.datetime.now() - start_time).seconds < 5 :
# 		print("Already crawled todays file, only one time crawl is allowed in a day")
# 	print("Execution time = %s" % (datetime.datetime.now() - start_time))   
# 	print("Today's crawl completed")



In [43]:
os.chdir('/Users/ujjawalnarayan/Desktop/govt_crawl/history_crawl/')

In [54]:
#grabbing data of all open docket
start_time = datetime.datetime.now() 
print("grabbing data for history")
temp_file_name_with_path = temp_open_close_all_docket_data_downloader("history_docket", base_link)
print("completed")
print("Execution time = %s" % (datetime.datetime.now() - start_time)) 

grabbing data for history
https://www.gao.gov/search?f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01
https://www.gao.gov/search??f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&keyword=&f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&page=0
working on link no 1 of total links 312 0.3205128205128205 % completed
https://www.gao.gov/search??f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&keyword=&f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&page=1
working on link no 2 of total links 312 0.641025641025641 % completed
https://www.gao.gov/search??f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&keyword=&f%5B0%5D=content_type%3ABid%20Protest%20Decision&f%5B1%5D=date%3Astart%2B2000-01-01&page=2
working on link no 3 of total links 312 0.9615384615384616 % completed
https://www.gao

In [ ]:
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import requests
import datetime
import time
import os
import json
import glob
import random
import copy

In [ ]:
with open("history_docket_temp_data.json","r") as f :
    all_data = [json.loads(i) for i in f.readlines() if json.loads(i) != False]

In [ ]:
for single_data in all_data :
    new_dict = copy.deepcopy(single_data)
    decision_highlights_soupe = soup(single_data["highlights_tags"],"html.parser")
    decision_highlights_raw_text = decision_highlights_soupe.findAll("div",{"class" : "field__item"})[0].text.strip()
    new_dict["decision_highlights_raw_text"] = decision_highlights_raw_text
    
    #date and file no extraction logic
    if len(single_data["file_no_and_date"].split("\n\n")) > 1 :
        new_dict["File no(s)"] = single_data["file_no_and_date"].split("\n\n")[0].strip()
        new_dict["Date"] = single_data["file_no_and_date"].split("\n\n")[1].strip()
    else :
        new_dict["Date"] = single_data["file_no_and_date"].split("\n\n")[0].strip()
        new_dict["File no(s)"] = ""
    
    #decision extraction 

    if len(decision_highlights_soupe.findAll("div",{"class" : "status highlighted-status"})) == 0 and len(decision_highlights_soupe.findAll("p")) == 1 :
        new_dict["Decision"] = ""
    elif len(decision_highlights_soupe.findAll("div",{"class" : "status highlighted-status"})) == 0 and len(decision_highlights_soupe.findAll("p")) == 2:
        new_dict["Decision"] = decision_highlights_soupe.findAll("p")[1].text.strip()
    elif len(decision_highlights_soupe.findAll("div",{"class" : "status highlighted-status"})) > 0 :
        decision = decision_highlights_soupe.findAll("div",{"class" : "status highlighted-status"})[0].text.strip()
        new_dict["Decision"] = decision
    else :
        new_dict["Decision"] = ""
    
        
        
    
    new_dict["protestor"] = decision_highlights_raw_text.split(",")[0]
    #print(new_dict["protestor"])
    print(decision_highlights_raw_text)
    

In [ ]:
single_data = all_data[0]

In [ ]:
new_dict = copy.deepcopy(single_data)
decision_highlights_soupe = soup(single_data["highlights_tags"],"html.parser")
decision_highlights_raw_text = decision_highlights_soupe.findAll("div",{"class" : "field__item"})[0].text.strip()
new_dict["decision_highlights_raw_text"] = decision_highlights_raw_text
new_dict["protestor"] = decision_highlights_raw_text.split(", protests")[0]

In [ ]:
single_data["file_no_and_date"].split("\n\n")

In [6]:
all_links = available_search_result_page_links(base_link)

In [14]:
docket_page_links = docket_page_links_from_search_page(all_links[0]) 

In [15]:
docket_page_links

['https://www.gao.gov/products/b-420227',
 'https://www.gao.gov/products/b-420110%2Cb-420111',
 'https://www.gao.gov/products/b-420038',
 'https://www.gao.gov/products/b-420021%2Cb-420021.2%2Cb-420021.3',
 'https://www.gao.gov/products/b-419838%2Cb-419838.2',
 'https://www.gao.gov/products/b-419973',
 'https://www.gao.gov/products/b-420030%2Cb-420030.2',
 'https://www.gao.gov/products/b-420013%2Cb-420013.2',
 'https://www.gao.gov/products/b-419893.8%2Cb-419893.9',
 'https://www.gao.gov/products/b-419675.2%2Cb-419675.3%2Cb-419675.4',
 'https://www.gao.gov/products/b-418255.5%2Cb-418255.6',
 'https://www.gao.gov/products/b-419894.2',
 'https://www.gao.gov/products/b-419560.6',
 'https://www.gao.gov/products/b-419972',
 'https://www.gao.gov/products/b-419984',
 'https://www.gao.gov/products/b-419557.2%2Cb-419557.3%2Cb-419557.4',
 'https://www.gao.gov/products/b-419997',
 'https://www.gao.gov/products/b-419966',
 'https://www.gao.gov/products/b-419000.3',
 'https://www.gao.gov/products/b-4

In [38]:
extract_data_from_docket_page(docket_page_links[5])

{'docket_page_link': 'https://www.gao.gov/products/b-419973',
 'highlights_tags': '<div class="clearfix text-formatted field field--name-product-highlights-custom field--type-text-long field--label-above quickedit-field">\n<header class="field__label">Highlights</header>\n<div class="field__item">\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN" "http://www.w3.org/TR/REC-html40/loose.dtd">\n\n<html><body><p>Davos V. Francois (Francois), of Miramar, Florida, protests its nonselection under solicitation (Soln.) No. PSC-21-044-INL, issued by the Department of State (DOS) for a contracting officer representative (COR) position in Haiti. The protester contends it was improperly penalized for not having an active certification. The protester argues that this error resulted in its nonselection.</p>\n</body></html>\n<div class="status highlighted-status">\n        We deny the protest.\n      </div>\n</div>\n</div>',
 'decision_tags': '<div class="js-endpoint-view-decision field f

In [17]:
docket_page_links[5]

'https://www.gao.gov/products/b-419973'

In [25]:
browse_link = html_data_grabber(docket_page_links[5])